# --- Day 16: Proboscidea Volcanium ---

https://adventofcode.com/2022/day/16

## Get Input Data

In [1]:
import re

def parse_input(filename):
    """Parse input data for today's puzzle.
    
    Parameters
    ----------
    filename : str
        The name of the *.txt file in the inputs/ directory.
    
    Returns
    -------
    graph : dict
    flow_rates : dict
    """
    graph = {}
    flow_rates = {}

    pattern = r"Valve ([A-Z]{2}) .+=(\d+); .+ valves? (.+)"
    file_str = open(f"../inputs/{filename}.txt").read()

    for node, flow_rate, neighbors in re.findall(pattern, file_str):
        graph[node] = {}
        for neighbor in neighbors.split(", "):
            graph[node][neighbor] = 1
                
        flow_rates[node] = int(flow_rate)

    return graph, flow_rates

In [2]:
test_valves_graph, test_flow_rates = parse_input("test_valves")
test_valves_graph

{'AA': {'DD': 1, 'II': 1, 'BB': 1},
 'BB': {'CC': 1, 'AA': 1},
 'CC': {'DD': 1, 'BB': 1},
 'DD': {'CC': 1, 'AA': 1, 'EE': 1},
 'EE': {'FF': 1, 'DD': 1},
 'FF': {'EE': 1, 'GG': 1},
 'GG': {'FF': 1, 'HH': 1},
 'HH': {'GG': 1},
 'II': {'AA': 1, 'JJ': 1},
 'JJ': {'II': 1}}

In [3]:
test_flow_rates

{'AA': 0,
 'BB': 13,
 'CC': 2,
 'DD': 20,
 'EE': 3,
 'FF': 0,
 'GG': 0,
 'HH': 22,
 'II': 0,
 'JJ': 21}

In [ ]:
def solve_part1(filename):
    valves_graph, flow_rates = parse_input(filename)

    # Only want to consider valves that have positive flow rates
    worthwhile_valves = [v for v in flow_rates if flow_rates[v] > 0]

    min_dists = fw(valves_graph) # Maybe use BFS for each start/end?

    all_paths = find_all_paths()
    releases = calc_releases(all_paths)

    max_release = max(releases)

    return max_release

In [5]:
valves_graph, flow_rates = parse_input("valves")

In [6]:
dict(list(valves_graph.items())[:10])

{'XB': {'WZ': 1, 'LE': 1},
 'BM': {'PL': 1, 'RI': 1},
 'GC': {'HN': 1, 'IT': 1},
 'RM': {'ZQ': 1, 'YL': 1},
 'ZM': {'SN': 1, 'KE': 1, 'UW': 1, 'MY': 1, 'GW': 1},
 'UH': {'HM': 1, 'HN': 1},
 'GW': {'LE': 1, 'ZM': 1},
 'HN': {'UW': 1, 'UH': 1, 'GL': 1, 'WZ': 1, 'GC': 1},
 'VT': {'ZD': 1, 'PE': 1},
 'VI': {'JS': 1, 'AA': 1}}

In [7]:
dict(list(flow_rates.items())[:10])

{'XB': 0,
 'BM': 0,
 'GC': 0,
 'RM': 0,
 'ZM': 5,
 'UH': 0,
 'GW': 0,
 'HN': 19,
 'VT': 0,
 'VI': 0}

In [8]:
worthwhile_valves = [v for v in flow_rates if flow_rates[v] > 0]
worthwhile_valves

['ZM',
 'HN',
 'YL',
 'JI',
 'ZD',
 'XD',
 'ES',
 'IY',
 'QY',
 'SM',
 'QR',
 'ZQ',
 'PL',
 'PE',
 'LE']

## Part 1
---

What is the decision to make at each minute?

These are the state variables:
1. `current_valve` : Where in the graph we are
2. `time_left` : Countdown in minutes from 30
3. `open_valves` : An array keeping track of which of the `worthwhile_valves` are open (order matters, but that's captured in the `pressure_release`)
4. `closed_valves` : An array keeyping track of which of the `worthwhile_vales` are still closed
5. `pressure_release` : The amount of pressure that would be released if 

At each minute, have to decide:
* Open current valve?
* Move to another valve?
* Stay put? (All `worthwhile_valves` are open already, so there's nothing worth doing)

While the test data is relatively small, the actual input data is significantly larger.  
The test data have only 6 `worthwhile_valves`, but the actual input data have 15 `worthwhile_valves`.

While that doesn't seem like a big difference, it is!

That's because the number of possible permutations explodes as $n$ increases.  
The number of permutations of an array with $n$ elements is $n!$

`math.factorial(6)` is 720  
`math.factorial(15)` is ~1.3 trillion





In [9]:
import math
print(f" 6! is equal to: {math.factorial(6)}")
print(f"15! is equal to: {math.factorial(15)}")

 6! is equal to: 720
15! is equal to: 1307674368000


In [10]:
from collections import defaultdict
from itertools import product

def fw(graph):
    """See algorithms/floyd_warshall.ipynb for comments/details."""
    dist = defaultdict(lambda: float("inf"))

    for node, neighbors in graph.items():
        dist[(node, node)] = 0
        for neighbor, edge_weight in neighbors.items():
            dist[(node, neighbor)] = edge_weight

    for k, i, j in product(graph.keys(), repeat=3):
        dist[(i, j)] = min(dist[(i, j)], dist[(i, k)] + dist[(k, j)])

    return dict(dist)

In [11]:
test_min_dists = fw(test_valves_graph)
dict(list(test_min_dists.items())[:5])

{('AA', 'AA'): 0,
 ('AA', 'DD'): 1,
 ('AA', 'II'): 1,
 ('AA', 'BB'): 1,
 ('BB', 'BB'): 0}

In [12]:
min_dists = fw(valves_graph)
dict(list(min_dists.items())[:5])

{('XB', 'XB'): 0,
 ('XB', 'WZ'): 1,
 ('XB', 'LE'): 1,
 ('BM', 'BM'): 0,
 ('BM', 'PL'): 1}

In [13]:
from functools import cache

In [97]:
def find_all_perms(curr_valve, time_left=30, perm='', release=0):

    perms = []
    perm += curr_valve + ","

    if len(perm) > 3:
        prev_valve = perm[-6:-4]
        time_left -= (min_dists[(prev_valve, curr_valve)] + 1)

    # Base case
    if time_left <= 0:
        return perms
    
    else:
        release += time_left * flow_rates[curr_valve]
        perms += [release]

        for valve in worthwhile_valves:
            if valve not in perm:
                perms += find_all_perms(valve, time_left, perm, release)
            
        return perms

In [98]:
all_perms = find_all_perms("AA")
print(min(all_perms), max(all_perms), len(all_perms))

0 2265 136465


In [39]:
# Experimenting
cache = {}
def find_all_perms(curr_valve, time_left=30, perm='', release=0):

    perms = []

    for valve in worthwhile_valves:
        if valve not in [perm[i:i+2] for i in range(0, int(len(perm)), 2)]:

            perm += valve
            
            if perm in cache:
                perms += [cache[perm]]
                return perms

            time_left -= (min_dists[(curr_valve, valve)] + 1)
            if time_left <= 0:
                return perms
            else:
                release += time_left * flow_rates[curr_valve]
                perms += [release]
                cache[perm] = release

                perms += find_all_perms(valve, time_left, perm, release)

    return perms

In [40]:
len(find_all_perms("AA"))

9

In [38]:
len(cache)

136470

In [66]:
# on test inputs
def find_all_paths(curr_valve, time_left=30, path='', release=0):

    paths = []
    path += curr_valve

    if len(path) > 2:
        prev_valve = path[-4:-2]
        time_left -= (test_min_dists[(prev_valve, curr_valve)] + 1)
        release += time_left * test_flow_rates[curr_valve]

    paths += [release]

    # Base case
    if time_left <= 2:
        return paths
        
    for valve in test_worthwhile_valves:
        if valve not in [path[i:i+2] for i in range(0, int(len(path)), 2)]:
            paths += find_all_paths(valve, time_left, path, release)

    return paths

all_paths = find_all_paths("AA")
print(min(all_paths), max(all_paths), len(all_paths))


0 1651 1957


### Run on Test Data

In [ ]:
max_cum_pressure_release(test_valves_graph, test_flow_rates) # == 1651

### Run on Input Data

## Part 2
---

### Run on Test Data

### Run on Input Data